### Importing libraries

In [1]:
import pandas as pd
from collections import defaultdict
import math

In [2]:
file_path1 = 'TestData/amblyopiaStudy Data ExportSanaTest.tsv'
file_path2 = 'TestData/amblyopiaStudy MetricsSanaTest.tsv'

dataExport = pd.read_csv(file_path1, sep='\t')
dataMetric = pd.read_csv(file_path2, sep='\t')

### Extracting features

In [3]:
dataExport = dataExport[["Eyetracker timestamp", "Event", "Event value", "Gaze point X (MCSnorm)", "Gaze point Y (MCSnorm)", "Eye movement type", 
    'AOI hit [SaccadeStimulus-Cat - bottomLeftCat1]',
       'AOI hit [SaccadeStimulus-Cat - bottomRightCat]',
       'AOI hit [SaccadeStimulus-Cat - bottomRightCat2]',
       'AOI hit [SaccadeStimulus-Cat - centerCat]',
       'AOI hit [SaccadeStimulus-Cat - centerCat2]',
       'AOI hit [SaccadeStimulus-Cat - topCenterCat]',
       'AOI hit [SaccadeStimulus-Cat - topLeftCat]',
       'AOI hit [SaccadeStimulus-Cat - topRightCat]',
       'AOI hit [SaccadeStimulus-Sloth - bottomLeftSloth]',
       'AOI hit [SaccadeStimulus-Sloth - bottomRightSloth]',
       'AOI hit [SaccadeStimulus-Sloth - centerSloth]',
       'AOI hit [SaccadeStimulus-Sloth - topCenterSloth]', 
        'Gaze point left X (MCSnorm)', 'Gaze point left Y (MCSnorm)', 'Gaze point right X (MCSnorm)', 'Gaze point right Y (MCSnorm)']]
dataExportWithoutCalibration = dataExport[6080:]
                              
mask = dataExportWithoutCalibration["Eye movement type"] == "Saccade"
dfSaccade = dataExportWithoutCalibration[mask]
dfSaccade = dfSaccade.dropna(subset=["Gaze point X (MCSnorm)", "Gaze point Y (MCSnorm)"], inplace=False)

#highest value means HIT

### Identify the first saccade for each AOI

In [4]:
AOI_list = ['AOI hit [SaccadeStimulus-Cat - bottomLeftCat1]',
       'AOI hit [SaccadeStimulus-Cat - bottomRightCat]',
       'AOI hit [SaccadeStimulus-Cat - bottomRightCat2]',
       'AOI hit [SaccadeStimulus-Cat - centerCat]',
       'AOI hit [SaccadeStimulus-Cat - centerCat2]',
       'AOI hit [SaccadeStimulus-Cat - topCenterCat]',
       'AOI hit [SaccadeStimulus-Cat - topLeftCat]',
       'AOI hit [SaccadeStimulus-Cat - topRightCat]',
       'AOI hit [SaccadeStimulus-Sloth - bottomLeftSloth]',
       'AOI hit [SaccadeStimulus-Sloth - bottomRightSloth]',
       'AOI hit [SaccadeStimulus-Sloth - centerSloth]',
       'AOI hit [SaccadeStimulus-Sloth - topCenterSloth]']

entrySaccades = defaultdict(int)

for i in AOI_list: 
    print(i)
    print(dfSaccade[i].value_counts().idxmax())
    first_negative_index = (dfSaccade[i] == 0).idxmax()
    entrySaccades[i] = first_negative_index



AOI hit [SaccadeStimulus-Cat - bottomLeftCat1]
0.0
AOI hit [SaccadeStimulus-Cat - bottomRightCat]
0.0
AOI hit [SaccadeStimulus-Cat - bottomRightCat2]
-1.0
AOI hit [SaccadeStimulus-Cat - centerCat]
0.0
AOI hit [SaccadeStimulus-Cat - centerCat2]
-1.0
AOI hit [SaccadeStimulus-Cat - topCenterCat]
-1.0
AOI hit [SaccadeStimulus-Cat - topLeftCat]
0.0
AOI hit [SaccadeStimulus-Cat - topRightCat]
0.0
AOI hit [SaccadeStimulus-Sloth - bottomLeftSloth]
-1.0
AOI hit [SaccadeStimulus-Sloth - bottomRightSloth]
-1.0
AOI hit [SaccadeStimulus-Sloth - centerSloth]
0.0
AOI hit [SaccadeStimulus-Sloth - topCenterSloth]
0.0


### Finding amplitude for both eyes, right eye, left eye

In [5]:
df_reset = dfSaccade.reset_index()
df_reset['index'] = df_reset['index'].astype(int)

df_reset_orginal = dataExportWithoutCalibration.reset_index()
df_reset_orginal['index'] = df_reset_orginal['index'].astype(int)

amplitudeBothEyesDict = defaultdict(int)
amplitudeLeftEyeDict = defaultdict(int)
amplitudeRightEyeDict = defaultdict(int)


for key,value in entrySaccades.items(): 
    row1 = df_reset[df_reset['index'] == value]
    x1 = row1['Gaze point X (MCSnorm)'].iloc[0]
    y1 = row1['Gaze point Y (MCSnorm)'].iloc[0]
    new_value = value -1
    row2 = df_reset_orginal[df_reset_orginal['index'] == new_value]
    x2 = row2.loc[:,'Gaze point X (MCSnorm)'].iloc[0]
    y2 = row2.loc[:,'Gaze point Y (MCSnorm)'].iloc[0]
    deltaX = x1-x2
    deltaY = y1-y2
    distance = math.sqrt((deltaX**2) + (deltaY**2))
    amplitudeBothEyesDict[key] = distance
    
    x1 = row1['Gaze point left X (MCSnorm)'].iloc[0]
    y1 = row1['Gaze point left Y (MCSnorm)'].iloc[0]
    x2 = row2.loc[:,'Gaze point left X (MCSnorm)'].iloc[0]
    y2 = row2.loc[:,'Gaze point left Y (MCSnorm)'].iloc[0]
    deltaX = x1-x2
    deltaY = y1-y2
    distance = math.sqrt((deltaX**2) + (deltaY**2))
    amplitudeLeftEyeDict[key] = distance
    
    x1 = row1['Gaze point right X (MCSnorm)'].iloc[0]
    y1 = row1['Gaze point right Y (MCSnorm)'].iloc[0]
    x2 = row2.loc[:,'Gaze point right X (MCSnorm)'].iloc[0]
    y2 = row2.loc[:,'Gaze point right Y (MCSnorm)'].iloc[0]
    deltaX = x1-x2
    deltaY = y1-y2
    distance = math.sqrt((deltaX**2) + (deltaY**2))
    amplitudeRightEyeDict[key] = distance
    

    

### Extracting relevant features from data metric

In [6]:
dataMetricSaccades = dataMetric.iloc[:,list(range(520, dataMetric.shape[1]))]


In [7]:
dataMetricSaccades[['Number_of_saccades_in_AOI.bottomLeftCat1',
       'Number_of_saccades_in_AOI.bottomLeftSloth',
       'Number_of_saccades_in_AOI.bottomRightCat',
       'Number_of_saccades_in_AOI.bottomRightCat2',
       'Number_of_saccades_in_AOI.bottomRightSloth',
       'Number_of_saccades_in_AOI.centerCat',
       'Number_of_saccades_in_AOI.centerCat2',
       'Number_of_saccades_in_AOI.centerSloth',
       'Number_of_saccades_in_AOI.topCenterCat',
       'Number_of_saccades_in_AOI.topCenterSloth',
       'Number_of_saccades_in_AOI.topLeftCat',
       'Number_of_saccades_in_AOI.topRightCat',
       'Number_of_saccades_in_AOI.topRightSloth',
       'Number_of_saccades_in_AOI.topRightSloth1',
       'Time_to_entry_saccade.bottomLeftCat1']]

,Number_of_saccades_in_AOI.bottomLeftCat1,Number_of_saccades_in_AOI.bottomLeftSloth,Number_of_saccades_in_AOI.bottomRightCat,Number_of_saccades_in_AOI.bottomRightCat2,Number_of_saccades_in_AOI.bottomRightSloth,Number_of_saccades_in_AOI.centerCat,Number_of_saccades_in_AOI.centerCat2,Number_of_saccades_in_AOI.centerSloth,Number_of_saccades_in_AOI.topCenterCat,Number_of_saccades_in_AOI.topCenterSloth,Number_of_saccades_in_AOI.topLeftCat,Number_of_saccades_in_AOI.topRightCat,Number_of_saccades_in_AOI.topRightSloth,Number_of_saccades_in_AOI.topRightSloth1,Time_to_entry_saccade.bottomLeftCat1
0,0.0,NaN,1.0,0.0,NaN,2.0,0.0,NaN,0.0,NaN,0.0,0.0,NaN,NaN,NaN
1,NaN,4.0,NaN,NaN,0.0,NaN,NaN,4.0,NaN,1.0,NaN,NaN,2.0,2.0,NaN
2,0.0,4.0,1.0,0.0,0.0,2.0,0.0,4.0,0.0,1.0,0.0,0.0,2.0,2.0,NaN


In [8]:
dataMetricSaccades['Time_to_entry_saccade.centerCat2']

0   NaN
1   NaN
2   NaN
Name: Time_to_entry_saccade.centerCat2, dtype: float64